In [ ]:
import os
import sys
sys.path.append(os.path.abspath(os.path.pardir))
sys.path.append(os.path.abspath(os.path.pardir)+'/voltaml')

from argparse import ArgumentParser

import torch
from voltaml.compile import VoltaGPUCompiler
from voltaml.inference import gpu_performance
import torchvision
from voltaml.models.common import DetectMultiBackend
from voltaml.models.yolov6.utils.checkpoint import load_checkpoint
from voltaml.models.yolov6.layers.common import DetectBackend

import warnings
warnings.filterwarnings('ignore')
# os.chdir('../voltaml')

In [ ]:
# Download a pretrained model
# import torch
torch.hub.download_url_to_file('https://github.com/meituan/YOLOv6/releases/download/0.2.0/yolov6s.pt', 'yolov6s.pt')

### Load models

In [ ]:
torch_model_dir = 'yolov6s.pt'
device = torch.device(0)

In [ ]:
#Load PyTorch model
model = load_checkpoint(torch_model_dir, map_location=device, inplace=True, fuse=True)  # load FP32 model

## Set parameters for FP16

In [ ]:
input_shape = (4,3,640,640)
precision = 'fp16'
compiled_model_dir = 'yolov6m_16_640.engine' ## Compiled model directory
is_yolo = True
input_name = 'images'
output_name = 'output'
simplify = True

### Compile Model

In [ ]:
compiler = VoltaGPUCompiler(
    model=model,
    output_dir=compiled_model_dir,
    input_shape=input_shape,
    precision=precision,
    input_name=input_name,
    output_name=output_name,
    simplify=simplify
    
)

compiled_model = compiler.compile()

In [ ]:
gpu_performance(compiled_model_dir, model, input_shape=input_shape)